# Harmonization
## Erica Simon, 02/02/24
## Purpose: Use the aneris tool to harmonize emisisons datasets
- Historical (1750-2022) and projected (2020-2100) emissions datasets
    - Historical dataset is product of PRIMAP, CEDS, GCP, & GFED. More info in `hist_emis_clean`
    - Projected datasets are outputs of GCAM 6.0 NGFS, MESSAGEix-GLOBIOM, and REMIND-MAgPIE 3.2-4.6. They were last updated in November 2023 and reflect commitments made until March 2023
    
    
Credit: 
- Gidden, M. (2017). gidden/aneris: Initial release version for harmonization paper (v0.1.0). Zenodo. https://doi.org/10.5281/ZENODO.802832
- https://github.com/chrisroadmap/fair-calibrate/blob/main/input/fair-2.1.3/v1.4/all-2022/calibration/12_harmonize-emissions.py


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import aneris
import aneris.convenience
from tqdm.auto import tqdm

import datetime
import os
import warnings

## Import datasets

In [2]:
GCAM_df = pd.read_csv('../outputs/GCAM_cleaned.csv')
MSG_df = pd.read_csv('../outputs/MSG_cleaned.csv')
REM_df = pd.read_csv('../outputs/REM_cleaned.csv')
hist_df = pd.read_csv('../outputs/hist_emis_cleaned.csv')

## Harmonize
This section of code adapted from https://github.com/chrisroadmap/fair-calibrate/blob/main/input/fair-2.1.3/v1.4/all-2022/calibration/12_harmonize-emissions.py


### set overrides

In [3]:
# Harmonization overrides - use same as RCMIP
overrides = pd.DataFrame(
    [
        {
            "method": "reduce_ratio_2150_cov",
            "variable": "Emissions|C2F6",
        },  # high historical variance (cov=16.2)
        {
            "method": "reduce_ratio_2150_cov",
            "variable": "Emissions|C6F14",
        },  # high historical variance (cov=15.4)
        {
            "method": "reduce_ratio_2150_cov",
            "variable": "Emissions|CF4",
        },  # high historical variance (cov=11.2)
        {
            "method": "reduce_ratio_2150_cov",
            "variable": "Emissions|CO",
        },  # high historical variance (cov=15.4)
        {
            "method": "reduce_ratio_2080",
            "variable": "Emissions|CO2",
        },  # always ratio method by choice
        {
            "method": "reduce_offset_2150_cov",
            "variable": "Emissions|CO2 AFOLU",
        },  # high historical variance, but using offset method to prevent diff from
        # increasing when going negative rapidly (cov=23.2)
        {
            "method": "reduce_ratio_2080",  # always ratio method by choice
            "variable": "Emissions|CO2 FFI",
        },
        #     {'method': 'default_aneris_tree', 'variable': 'Emissions|CH4'},
        # depending on the decision tree in aneris/method.py
        {
            "method": "constant_ratio",
            "variable": "Emissions|HFC-125",
        },  # minor f-gas with low model reporting confidence
        {
            "method": "constant_ratio",
            "variable": "Emissions|HFC-134a",
        },  # minor f-gas with low model reporting confidence
        {
            "method": "constant_ratio",
            "variable": "Emissions|HFC-143a",
        },  # minor f-gas with low model reporting confidence
        {
            "method": "constant_ratio",
            "variable": "Emissions|HFC-227ea",
        },  # minor f-gas with low model reporting confidence
        {
            "method": "constant_ratio",
            "variable": "Emissions|HFC-23",
        },  # minor f-gas with low model reporting confidence
        {
            "method": "constant_ratio",
            "variable": "Emissions|HFC-32",
        },  # minor f-gas with low model reporting confidence
        {
            "method": "constant_ratio",
            "variable": "Emissions|HFC-4310mee",
        },  # minor f-gas with low model reporting confidence
        #     {'method': 'default_aneris_tree', 'variable': 'Emissions|N2O'},
        # depending on the decision tree in aneris/method.py
        #     {'method': 'default_aneris_tree', 'variable': 'Emissions|NH3'},
        # depending on the decision tree in aneris/method.py
        #     {'method': 'default_aneris_tree', 'variable': 'Emissions|NOx'},
        # depending on the decision tree in aneris/method.py
        {
            "method": "reduce_ratio_2150_cov",
            "variable": "Emissions|OC",
        },  # high historical variance (cov=18.5)
        {
            "method": "constant_ratio",
            "variable": "Emissions|SF6",
        },  # minor f-gas with low model reporting confidence
        #     {'method': 'default_aneris_tree', 'variable': 'Emissions|Sulfur'},
        # depending on the decision tree in aneris/method.py
        {
            "method": "reduce_ratio_2150_cov",
            "variable": "Emissions|VOC",
        },  # high historical variance (cov=12.0)
    ]
)

we need to remove the override for HFC-23 in the MESSAGE IAM because has all emissions at 0 for this specie

In [4]:
MSG_overrides = overrides[~(overrides.variable == 'Emissions|HFC-23')]

### run harmonizer

In [5]:
progress = os.getenv("PROGRESS", "False").lower() in ("true", "1", "t")

In [ ]:
for _, msdf in tqdm(GCAM_df.groupby(["Model", "Scenario"]), disable=1 - progress):
    print(msdf)

In [6]:
def harmonize(hist, future, harm_yr, overrides):

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        scenarios_harmonized = [
            aneris.convenience.harmonise_all(
                msdf,
                history=hist,
                harmonisation_year=harm_yr,
                overrides=overrides,
            )
            for _, msdf in tqdm(future.groupby(["Model", "Scenario"]), disable=1 - progress)
        ]

    scenarios_harmonized = pd.concat(scenarios_harmonized).reset_index()
    return scenarios_harmonized

In [7]:
GCAM_harm = harmonize(hist_df, GCAM_df, 2022, overrides)
MSG_harm = harmonize(hist_df, MSG_df, 2022, MSG_overrides)
REM_harm = harmonize(hist_df, REM_df, 2022, overrides)

TypeError: 'NoneType' object is not callable

### combine hist and future dfs into one harmonized dataset

In [ ]:
yrs = np.arange(1750, 2101, 1)
new_column_order = ['model', 'scenario', 'region', 'Variable', 'unit']

for yr in yrs:
    new_column_order.append(str(yr))

In [ ]:
def combine(hist_df, future_df):
    # rename to match case of hist df
    future_df = future_df.rename(columns={"variable": "Variable"})
    
    # merge hist and future
    df = pd.merge(future_df, hist_df, on='Variable', how='inner')
    
    # reorder columns to be in correct year order
    df.columns = df.columns.astype(str)
    df.drop(columns=['Model', 'Scenario', 'Region', 'Unit'], inplace=True)  # remove extra columns from hist
    df = df.iloc[:, :-1]  # remove duplicate 2022 column
    df = df[new_column_order]
    
    # update column names for uppercase consistency
    df = df.rename(columns={"model": "Model", "scenario": "Scenario", "region": "Region", "unit": "Unit"})
    
    return df

In [ ]:
GCAM_full = combine(hist_df, GCAM_harm)
MSG_full = combine(hist_df, MSG_harm)
REM_full = combine(hist_df, REM_harm)

### save harmonized data

In [ ]:
GCAM_full.to_csv('../outputs/GCAM_harm.csv', index=False)
MSG_full.to_csv('../outputs/MSG_harm.csv', index=False)
REM_full.to_csv('../outputs/REM_harm.csv', index=False)

## Visualize

In [ ]:
def plot_harm(hist, future, harm_ized, var, scen):
    histt = hist.loc[hist.Variable == 'Emissions|'+var].values[0][255:]
    raw = future.loc[future.Variable == 'Emissions|'+var].loc[future.Scenario == scen].values[0][7:]
    harm = harm_ized.loc[harm_ized.variable == 'Emissions|'+var].loc[harm_ized.scenario == scen].values[0][5:]
    unit = hist.loc[hist.Variable == 'Emissions|'+var].values[0][4]
    mdl = harm_ized.model.values[0]
    
    plt.plot(np.arange(2000, 2023), histt)
    plt.plot(np.arange(2022, 2101), raw, label='unharmonized')
    plt.plot(np.arange(2022, 2101), harm, label='harmonized')
    plt.axvline(2022, linestyle='--', color='grey', alpha=0.5)
    plt.xlabel('Year')
    plt.ylabel(unit)
    plt.title('Harmonized '+var+' Emissions\n'+scen+' Scenario\n'+mdl)
    plt.legend()
    plt.show()

In [ ]:
def plot_harm_ax(hist, future, harm_ized, var, scen, ax, maxes):
    histt = hist.loc[hist.Variable == 'Emissions|'+var].values[0][255:]
    raw = future.loc[future.Variable == 'Emissions|'+var].loc[future.Scenario == scen].values[0][7:]
    harm = harm_ized.loc[harm_ized.variable == 'Emissions|'+var].loc[harm_ized.scenario == scen].values[0][5:]
    unit = hist.loc[hist.Variable == 'Emissions|'+var].values[0][4]
    mdl = harm_ized.model.values[0]
    
    # find maximum value on plot
    max_val = max(raw.max(), harm.max())
    maxes.append(max_val)
    
    
    ax.plot(np.arange(2000, 2023), histt)
    ax.plot(np.arange(2022, 2101), raw)
    ax.plot(np.arange(2022, 2101), harm)
    ax.axvline(2022, linestyle='--', color='grey', alpha=0.5)
    # ax.set_ylim(0, 450)
    ax.set_title(mdl)

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(15, 5))

maxes = []

for i, (hist, future, harm) in enumerate([(hist_df, GCAM_df, GCAM_harm), (hist_df, MSG_df, MSG_harm), (hist_df, REM_df, REM_harm)]):
    plot_harm_ax(hist, future, harm, 'CH4', 'Current Policies', axs[i], maxes)
    
for i in range (len(axs)):
    axs[i].set_ylim(0, max(maxes))

plt.suptitle('CH4 Harmonized Emissions\nCurrent Policies Scenario', fontsize=15)
fig.text(-0.01, 0.45, 'Mt CH4/yr', va='center', rotation='vertical', fontsize=12)
fig.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(15, 5))

maxes = []

for i, (hist, future, harm) in enumerate([(hist_df, GCAM_df, GCAM_harm), (hist_df, MSG_df, MSG_harm), (hist_df, REM_df, REM_harm)]):
    plot_harm_ax(hist, future, harm, 'CO2 FFI', 'Current Policies', axs[i], maxes)

for i in range (len(axs)):
    axs[i].set_ylim(0, max(maxes))

plt.suptitle('CO2 FFI Harmonized Emissions\nCurrent Policies Scenario', fontsize=15)
fig.text(-0.01, 0.45, 'Gt CO2/yr', va='center', rotation='vertical', fontsize=12)
fig.tight_layout()
plt.show()

### check unit adjustment

In [ ]:
plot_harm(hist_df, GCAM_df, GCAM_harm, 'CO2 AFOLU', 'Current Policies')

In [ ]:
plot_harm(hist_df, GCAM_df, GCAM_harm, 'CO2 FFI', 'Current Policies')

In [ ]:
plot_harm(hist_df, GCAM_df, GCAM_harm, 'N2O', 'Current Policies')

In [ ]:
plot_harm(hist_df, GCAM_df, GCAM_harm, 'HFC-23', 'Current Policies')

In [ ]:
plot_harm(hist_df, MSG_df, MSG_harm, 'HFC-23', 'Current Policies')

In [ ]:
plot_harm(hist_df, REM_df, REM_harm, 'CH4', 'Current Policies')